# Preprocessing data with TensorFlow Transform

Trong quá trình xây dựng một mô hình Machine learning, các tập dữ liệu sử dụng cho việc train và test rất quan trọng, ảnh hưởng trực tiếp tới chất lượng mô hình. 
Tuy nhiên không phải tập dữ liệu nào cũng phù hợp cho việc xây dựng mô hình, với những tập dữ liệu này ta cần phải biển đổi và chuẩn hóa nó. \\
Thư viện TensorFlow Transform cung cấp các hàm có thể giúp việc chuẩn hóa dữ liệu trở nên dễ dàng hơn. 
Dưới đây là demo sử dụng thư viện TensorFlow Transform trong Preprocessing data.

### Cài đặt TensorFlow Transform

In [18]:
# Bỏ qua nếu đã cài đặt
# !pip install tensorflow-transform

In [2]:
# This cell is only necessary because packages were installed while python was
# running. It avoids the need to restart the runtime when running in Colab.
import pkg_resources
import importlib

importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.9/dist-packages/pkg_resources/__init__.py'>

### Import các thứ viện cần  thiết

Thư viện Tensorflow Transform cung cấp các hàm biến đổi dữ liệu tuy nhiên nó không trực tiếp biến đổi dữ liệu mà ta phải sử dụng các module khác như Apache Beam.

In [3]:
import math
import os
import pprint

import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
print('TF: {}'.format(tf.__version__))

import apache_beam as beam
print('Beam: {}'.format(beam.__version__))

import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
print('Transform: {}'.format(tft.__version__))

from tfx_bsl.public import tfxio
from tfx_bsl.coders.example_coder import RecordBatchToExamples

TF: 2.11.1
Beam: 2.46.0
Transform: 1.12.0


### Dữ liệu được sử dụng

Đây là bộ dữ liệu về dân số với 14 thuộc tính. 

In [4]:
!wget https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/census/adult.data
!wget https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/census/adult.test

train_path = './adult.data'
test_path = './adult.test'

--2023-04-09 17:48:09--  https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/census/adult.data
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.128, 142.250.153.128, 142.250.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [application/octet-stream]
Saving to: ‘adult.data.1’

adult.data.1        100%[===================>]   3.79M  5.86MB/s    in 0.6s    

2023-04-09 17:48:10 (5.86 MB/s) - ‘adult.data.1’ saved [3974305/3974305]

--2023-04-09 17:48:10--  https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/census/adult.test
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.128, 142.250.153.128, 142.250.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2003

In [5]:
pandas_train = pd.read_csv(train_path, header=None)

pandas_train.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Định nghĩa các thuộc tính cho dữ liệu và Schema

Ta sẽ tự định nghĩa các thuộc tính của dữ liệu.

In [6]:
CATEGORICAL_FEATURE_KEYS = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
]

NUMERIC_FEATURE_KEYS = [
    'age',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'education-num'
]

ORDERED_CSV_COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label'
]

LABEL_KEY = 'label'

In [7]:
RAW_DATA_FEATURE_SPEC = dict(
    [(name, tf.io.FixedLenFeature([], tf.string))
     for name in CATEGORICAL_FEATURE_KEYS] +
    [(name, tf.io.FixedLenFeature([], tf.float32))
     for name in NUMERIC_FEATURE_KEYS] + 
    [(LABEL_KEY, tf.io.FixedLenFeature([], tf.string))]
)

SCHEMA = tft.DatasetMetadata.from_feature_spec(RAW_DATA_FEATURE_SPEC).schema

### Sử dụng  các hàm của thư viện TFX Transform tạo function preprocessing_fn

Có thể thấy ở đoạn dữ liệu ban đầu, các thuộc tính vẫn chưa phù hợp để thực hiện các tác vụ như train hay các tác vụ khác, vì vậy ta cần phải chuẩn hóa dữ liệu: \\
- `tft.scale_to_0_1(inputs[key])`: Chuẩn hóa giá trị của các cột số trong phạm vi [0,1].
- `tft.compute_and_apply_vocabulary()`: Tạo vocabulary cho các cột không phải số (categorical features) và chuyển đổi chúng thành integer IDs.
- `tf.lookup.KeyValueTensorInitializer()`: Tạo một bảng ánh xạ giá trị chuỗi (string) sang index integer.
- `tf.lookup.StaticHashTable()`: Tạo một bảng ánh xạ tĩnh dùng để ánh xạ giá trị chuỗi sang index integer và sử dụng trong quá trình training.
- `tf.strings.regex_replace()`: Thay thế các ký tự trong giá trị chuỗi theo một regular expression cho trước.
- `tf.one_hot()`: Chuyển đổi giá trị đầu vào thành one-hot encoding.

In [8]:
NUM_OOV_BUCKETS = 2
TRANSFORMED_TRAIN_DATA_FILEBASE = 'train_transformed'
TRANSFORMED_TEST_DATA_FILEBASE = 'test_transformed'
EXPORTED_MODEL_DIR = 'exported_model_dir'
def preprocessing_fn(inputs):
  """Preprocess input columns into transformed columns."""
  # Since we are modifying some features and leaving others unchanged, we
  # start by setting `outputs` to a copy of `inputs.
  outputs = inputs.copy()

  # Scale numeric columns to have range [0, 1].
  for key in NUMERIC_FEATURE_KEYS:
    outputs[key] = tft.scale_to_0_1(inputs[key])

  # For all categorical columns except the label column, we generate a
  # vocabulary but do not modify the feature.  This vocabulary is instead
  # used in the trainer, by means of a feature column, to convert the feature
  # from a string to an integer id.
  for key in CATEGORICAL_FEATURE_KEYS:
    outputs[key] = tft.compute_and_apply_vocabulary(
        tf.strings.strip(inputs[key]),
        num_oov_buckets=NUM_OOV_BUCKETS,
        vocab_filename=key)

  # For the label column we provide the mapping from string to index.
  table_keys = ['>50K', '<=50K']
  with tf.init_scope():
    initializer = tf.lookup.KeyValueTensorInitializer(
        keys=table_keys,
        values=tf.cast(tf.range(len(table_keys)), tf.int64),
        key_dtype=tf.string,
        value_dtype=tf.int64)
    table = tf.lookup.StaticHashTable(initializer, default_value=-1)

  # Remove trailing periods for test data when the data is read with tf.data.
  # label_str  = tf.sparse.to_dense(inputs[LABEL_KEY])
  label_str = inputs[LABEL_KEY]
  label_str = tf.strings.regex_replace(label_str, r'\.$', '')
  label_str = tf.strings.strip(label_str)
  data_labels = table.lookup(label_str)
  transformed_label = tf.one_hot(
      indices=data_labels, depth=len(table_keys), on_value=1.0, off_value=0.0)
  outputs[LABEL_KEY] = tf.reshape(transformed_label, [-1, len(table_keys)])

  return outputs

### Thực hiện Preprocessing

#### Định nghĩa hàm thực thi `transform_data`. 
Hàm này sẽ thực hiện các công việc sau:
- Đọc dữ liệu huấn luyện từ tệp train_data_file bằng `tfxio.CsvTFXIO`.
- Biến đổi dữ liệu huấn luyện bằng `preprocessing_fn` và tạo `transform_fn` để sử dụng sau này cho việc biến đổi dữ liệu kiểm tra. Việc tạo `transform_fn` để đảm bảo việc áp dụng cùng một cách biến đổi cho cả hai tập dữ liệu.
- Ghi các bản ghi biến đổi của dữ liệu huấn luyện dưới dạng `TFRecord` sử dụng `beam.io.WriteToTFRecord`.
- Đọc dữ liệu kiểm tra từ tệp `test_data_file` bằng `tfxio.CsvTFXIO` và biến đổi dữ liệu kiểm tra bằng cách sử dụng `transform_fn`.
- Ghi các bản ghi biến đổi của dữ liệu kiểm tra dưới dạng `TFRecord` sử dụng `beam.io.WriteToTFRecord`.
- Lưu `transform_fn` để sử dụng cho việc biến đổi dữ liệu mới trong tương lai.


In [9]:
def transform_data(train_data_file, test_data_file, working_dir):
  """Transform the data and write out as a TFRecord of Example protos.

  Read in the data using the CSV reader, and transform it using a
  preprocessing pipeline that scales numeric data and converts categorical data
  from strings to int64 values indices, by creating a vocabulary for each
  category.

  Args:
    train_data_file: File containing training data
    test_data_file: File containing test data
    working_dir: Directory to write transformed data and metadata to
  """

  # The "with" block will create a pipeline, and run that pipeline at the exit
  # of the block.
  with beam.Pipeline() as pipeline:
    with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
      # Create a TFXIO to read the census data with the schema. To do this we
      # need to list all columns in order since the schema doesn't specify the
      # order of columns in the csv.
      # We first read CSV files and use BeamRecordCsvTFXIO whose .BeamSource()
      # accepts a PCollection[bytes] because we need to patch the records first
      # (see "FixCommasTrainData" below). Otherwise, tfxio.CsvTFXIO can be used
      # to both read the CSV files and parse them to TFT inputs:
      # csv_tfxio = tfxio.CsvTFXIO(...)
      # raw_data = (pipeline | 'ToRecordBatches' >> csv_tfxio.BeamSource())
      train_csv_tfxio = tfxio.CsvTFXIO(
          file_pattern=train_data_file,
          telemetry_descriptors=[],
          column_names=ORDERED_CSV_COLUMNS,
          schema=SCHEMA)

      # Read in raw data and convert using CSV TFXIO.
      raw_data = (
          pipeline |
          'ReadTrainCsv' >> train_csv_tfxio.BeamSource())

      # Combine data and schema into a dataset tuple.  Note that we already used
      # the schema to read the CSV data, but we also need it to interpret
      # raw_data.
      cfg = train_csv_tfxio.TensorAdapterConfig()
      raw_dataset = (raw_data, cfg)

      # The TFXIO output format is chosen for improved performance.
      transformed_dataset, transform_fn = (
          raw_dataset | tft_beam.AnalyzeAndTransformDataset(
              preprocessing_fn, output_record_batches=True))

      # Transformed metadata is not necessary for encoding.
      transformed_data, _ = transformed_dataset

      # Extract transformed RecordBatches, encode and write them to the given
      # directory.
      # TODO(b/223384488): Switch to `RecordBatchToExamplesEncoder`.
      _ = (
          transformed_data
          | 'EncodeTrainData' >>
          beam.FlatMapTuple(lambda batch, _: RecordBatchToExamples(batch))
          | 'WriteTrainData' >> beam.io.WriteToTFRecord(
              os.path.join(working_dir, TRANSFORMED_TRAIN_DATA_FILEBASE)))

      # Now apply transform function to test data.  In this case we remove the
      # trailing period at the end of each line, and also ignore the header line
      # that is present in the test data file.
      test_csv_tfxio = tfxio.CsvTFXIO(
          file_pattern=test_data_file,
          skip_header_lines=1,
          telemetry_descriptors=[],
          column_names=ORDERED_CSV_COLUMNS,
          schema=SCHEMA)
      raw_test_data = (
          pipeline
          | 'ReadTestCsv' >> test_csv_tfxio.BeamSource())

      raw_test_dataset = (raw_test_data, test_csv_tfxio.TensorAdapterConfig())

      # The TFXIO output format is chosen for improved performance.
      transformed_test_dataset = (
          (raw_test_dataset, transform_fn)
          | tft_beam.TransformDataset(output_record_batches=True))

      # Transformed metadata is not necessary for encoding.
      transformed_test_data, _ = transformed_test_dataset

      # Extract transformed RecordBatches, encode and write them to the given
      # directory.
      _ = (
          transformed_test_data
          | 'EncodeTestData' >>
          beam.FlatMapTuple(lambda batch, _: RecordBatchToExamples(batch))
          | 'WriteTestData' >> beam.io.WriteToTFRecord(
              os.path.join(working_dir, TRANSFORMED_TEST_DATA_FILEBASE)))

      # Will write a SavedModel and metadata to working_dir, which can then
      # be read by the tft.TFTransformOutput class.
      _ = (
          transform_fn
          | 'WriteTransformFn' >> tft_beam.WriteTransformFn(working_dir))

#### Sau khi định nghĩa xong hàm thực thi, tiến hành transform.



In [10]:
import tempfile
import pathlib

output_dir = os.path.join(tempfile.mkdtemp(), 'keras')

transform_data(train_path, test_path, output_dir)

Instructions for updating:
Use ref() instead.


Sau khi chuyển đổi, dữ liệu và các metadata được lưu trữ vào thư mục `output_dir`

### Kết quả thu được

Sử dụng đối tượng TFTransformOutput để truy cập thông tin về các phép biến đổi được áp dụng và các thông số được tính toán trên dữ liệu huấn luyện để chuẩn hóa các giá trị đầu vào của mô hình.

In [11]:
tf_transform_output = tft.TFTransformOutput(output_dir)

#### Thông tin về các thuộc tính sau khi chuyển đổi.

In [12]:
tf_transform_output.transformed_feature_spec()

{'age': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'capital-gain': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'capital-loss': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'education': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'education-num': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'hours-per-week': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 'label': FixedLenFeature(shape=[2], dtype=tf.float32, default_value=None),
 'marital-status': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'native-country': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'occupation': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'race': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'relationship': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 'sex': FixedLenFeature(shape=[], dtype=tf.int64,

#### Tạo thư mục để chứa data được transform và các metadata

In [13]:
!ls -l {output_dir}

total 15704
-rw-r--r-- 1 root root  5356449 Apr  9 17:49 test_transformed-00000-of-00001
-rw-r--r-- 1 root root 10712569 Apr  9 17:49 train_transformed-00000-of-00001
drwxr-xr-x 2 root root     4096 Apr  9 17:48 transformed_metadata
drwxr-xr-x 4 root root     4096 Apr  9 17:48 transform_fn


#### Dữ liệu đã được chuyển đổi và sẵn sàng cho training

Hàm lấy dữ liệu đã được transform để làm input cho training

In [14]:
def _make_training_input_fn(tf_transform_output, train_file_pattern,
                            batch_size):
  """An input function reading from transformed data, converting to model input.

  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    transformed_examples: Base filename of examples.
    batch_size: Batch size.

  Returns:
    The input data for training or eval, in the form of k.
  """
  def input_fn():
    return tf.data.experimental.make_batched_features_dataset(
        file_pattern=train_file_pattern,
        batch_size=batch_size,
        features=tf_transform_output.transformed_feature_spec(),
        reader=tf.data.TFRecordDataset,
        label_key=LABEL_KEY,
        shuffle=True)

  return input_fn

In [15]:
train_file_pattern = pathlib.Path(output_dir)/f'{TRANSFORMED_TRAIN_DATA_FILEBASE}*'

input_fn = _make_training_input_fn(
    tf_transform_output=tf_transform_output,
    train_file_pattern = str(train_file_pattern),
    batch_size = 10
)

Dữ liệu phục vụ training

In [16]:
for example, label in input_fn().take(1):
  break

pd.DataFrame(example)

,age,capital-gain,capital-loss,education,education-num,hours-per-week,marital-status,native-country,occupation,race,relationship,sex,workclass
0,0.260274,0.0,0.000000,0,0.533333,0.275510,2,0,7,0,3,1,3
1,0.246575,0.0,0.000000,1,0.600000,0.397959,3,0,3,1,1,0,0
2,0.479452,0.0,0.000000,0,0.533333,0.397959,0,0,2,0,0,0,0
3,0.164384,0.0,0.000000,1,0.600000,0.397959,1,0,3,1,2,0,0
4,0.027397,0.0,0.000000,0,0.533333,0.397959,1,0,1,0,2,0,2
5,0.383562,0.0,0.340909,1,0.600000,0.397959,0,0,1,0,0,0,1
6,0.136986,0.0,0.223600,8,0.200000,0.397959,2,0,6,0,3,0,0
7,0.356164,0.0,0.000000,0,0.533333,0.397959,3,1,8,0,3,0,0
8,0.150685,0.0,0.000000,2,0.800000,0.500000,0,0,5,0,0,0,0
9,0.150685,0.0,0.000000,0,0.533333,0.397959,2,0,9,0,2,0,0


In [17]:
label

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)>